In [2]:
import pandas as pd
import numpy as np

h1_seasons = ['2009', 
              '2010',
              '2009Pan',
              '2014',
              '2016']


for i in range(0, 100):
    df = pd.read_csv('simulated_data/simulation_by_birth_year_%s.csv'%i)
    max_birth_years = pd.read_csv('../raw_data/max_birth_years.csv', index_col='Season')

    h1_cases = df[df.Season.isin(h1_seasons)].copy()
    h3_cases = df[~df.Season.isin(h1_seasons)].copy()

    final_df = pd.DataFrame(columns=['birth_year',
                                           'I_obs_h1',
                                           'I_vac_h1',
                                           'I_obs_h3',
                                           'I_vac_h3',
                                           'season'])
    h1_agg = h1_cases.groupby(['Season', 'Birth_year', 'Vaccination_status']).sum().Count
    h3_agg = h3_cases.groupby(['Season', 'Birth_year', 'Vaccination_status']).sum().Count

    index = 0
    for season in set(df.Season):
        if season in h1_seasons:
            aggdf = h1_agg
            subtype = 'h1'
        else:
            aggdf = h3_agg
            subtype = 'h3'
        for byear in np.arange(1918, max_birth_years.loc[season, 'max_year'] + 1):
            try:
                vac_cases = aggdf.loc[(season, byear, 1), ]
            except KeyError:
                vac_cases = 0

            try:
                unvac_cases = aggdf.loc[(season, byear, 0), ]
            except KeyError:
                unvac_cases = 0

            final_df.loc[index, 'birth_year'] = byear
            final_df.loc[index, 'I_obs_%s'%subtype] = unvac_cases
            final_df.loc[index, 'I_vac_%s'%subtype] = vac_cases
            final_df.loc[index, 'season'] = season

            index += 1

    final_df = final_df.sort_values(['season', 'birth_year'])
    final_df.to_csv('simulated_data/standard_simulation_%s.csv'%i, index=False)
